In [ ]:
import serial

In [ ]:
ser = serial.Serial('COM13', 38400, timeout=1)

In [ ]:
import time
import binascii
def get_pt(pt):
    ser.flushInput()
    ser.write(("p" + binascii.hexlify(pt).decode("latin-1") + "\n").encode())
    time.sleep(0.01)
    return ser.read(34) #get response
def get_trace():
    ser.flushInput()
    ser.write("b\n".encode())
    time.sleep(0.01)
    return ser.read(254)
def get_key(key):
    ser.flushInput()
    ser.write(("k" + binascii.hexlify(key).decode("latin-1") + "\n").encode())

In [ ]:
import chipwhisperer as cw
project = cw.createProject("./projects/saml11.cwp", overwrite = True)
cw_project = cw.createProject("./projects/cw_saml11.cwp", overwrite = True)

In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"
# setup scope parameters
scope.gain.gain = 45
scope.adc.samples = 5000
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "high_z"
scope.io.tio2 = "high_z"
scope.io.hs2 = "clkgen"

In [ ]:
import copy
project = cw.createProject("./projects/saml11.cwp", overwrite = True)
cw_project = cw.createProject("./projects/cw_saml11.cwp", overwrite = True)
tc = copy.deepcopy(project.getTraceFormat())
cw_tc = cw_project.getTraceFormat()

In [ ]:
import chipwhisperer as cw
scope.adc.samples = 5796
from tqdm import trange
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
from chipwhisperer.capture.acq_patterns.tvlattest import AcqKeyTextPattern_TVLATTest
import numpy as np
import binascii
N = 500
traces = []
texts = []
ctexts = []
cw_traces = []
keys = []
project = cw.createProject("./projects/saml11.cwp", overwrite = True)
tc = copy.deepcopy(project.getTraceFormat())
#ktp = AcqKeyTextPattern_TVLATTest()
ktp = AcqKeyTextPattern_Basic()
ktp.initPair(N)
for i in trange(N):
    key, text = ktp.newPair()
    
    get_key(key)
    #time.sleep(0.01)
    
    #scope.arm()
        # wait for target to finish
    try:
        ct = bytearray(binascii.unhexlify(get_pt(text)[1:-1]))
    except Exception as e:
        continue
    if len(ct) < 16:
        continue #didn't get whole ct back
    keys.append(key)
    ##while target.isDone() is False and timeout:
        #timeout -= 1
        #time.sleep(0.01)

    #try:
    #    ret = scope.capture()
    #    if ret:
    #        print('Timeout happened during acquisition')
    #except IOError as e:
    #    print('IOError: %s' % str(e))
    
    
    tmp = binascii.unhexlify(get_trace()[1:-1].decode("latin-1"))
    trace = [0] * 63
    for j in range(int(len(tmp)/2)):
        trace[j] = tmp[j*2] | (tmp[j*2 + 1] << 8)
    
    cw_trace = scope.getLastTrace()
    cw_traces.append(cw_trace)
    traces.append(trace)
    texts.append(text)
    ctexts.append(ct)
    tc.addTrace(trace, text, ct, key)
    #cw_tc.addTrace(cw_trace, text, ct, key)
    
tc._isloaded = True
cw_tc._isloaded = True
project.traceManager().appendSegment(tc)
cw_project.traceManager().appendSegment(cw_tc)

In [ ]:
#tc._isloaded = True
tc.isLoaded()

In [ ]:
tc._isloaded = True
cw_tc._isloaded = True
project.traceManager().appendSegment(tc)
cw_project.traceManager().appendSegment(cw_tc)

In [ ]:
print(texts[0])
print(binascii.hexlify(ctexts[0]))
print(tc.getTextout(0))
for ct in ctexts:
    assert len(ct) == 16, "Len ct = {}".format(len(ct))
print(binascii.hexlify(keys[0]))

In [ ]:
from datetime import datetime
import copy

starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
cw_tc.config.setConfigFilename(cw_project.datadirectory + "traces/config_" + prefix + ".cfg")
cw_tc.config.setAttr("prefix", prefix)
cw_tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))
cw_tc.closeAll()
cw_project.save()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
xrange = range(len(traces[0]))
p.line(xrange, traces[1])
p.line(xrange, traces[0], line_color='red')
show(p)

In [ ]:
p = figure()
xrange = range(len(cw_traces[0]))
p.line(xrange, cw_traces[0])
show(p)

In [ ]:
import chipwhisperer as cw
from datetime import datetime
import copy

starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
tc.config.setConfigFilename(project.datadirectory + "traces/config_" + prefix + ".cfg")
tc.config.setAttr("prefix", prefix)
tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))
tc.closeAll()
project.save()

In [ ]:
tm = project.traceManager()
tm.numTraces()

In [ ]:
scope.dis()
target.dis()

In [ ]:
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit
from chipwhisperer.analyzer.attacks.models import AES128_8bit
from importlib import reload
AES128_8bit = reload(AES128_8bit)

#project = cw.openProject("projects/saml11.cwp")
tm = project.traceManager()
attack = CPA()
leak_model = AES128_8bit.AES128_8bit(AES128_8bit.LastroundHW)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(tm)
attack.setTraceStart(0)
attack.setTracesPerAttack(tm.numTraces())
attack.setIterations(1)
attack.setReportingInterval(50)
attack.setPointRange((0, -1))
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
import importlib
importlib.reload(cw)
cb = cw.getJupyterCallback(attack, 15)
attack_results = attack.processTracesNoGUI(cb)

In [ ]:
print(ctexts[0])

In [ ]:
plot_data = cw.analyzerPlots(attack_results)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

ret = plot_data.outputVsTime(11)

output_notebook()
p = figure()
rets = []
for i in range(0, 16):
    rets.append(plot_data.outputVsTime(i))

p = figure()
for ret in rets:
    p.line(ret[0], ret[2], line_color='green')
    p.line(ret[0], ret[3], line_color='green')
    
for ret in rets:
    p.line(ret[0], ret[1], line_color='red')


show(p)

In [ ]:
%matplotlib notebook
from matplotlib.pylab import *
import numpy as np
import scipy
import scipy.stats
fixedpy = [0xda, 0x39, 0xa3, 0xee, 0x5e, 0x6b, 0x4b, 0x0d, 0x32, 0x55, 0xbf, 0xef, 0x95, 0x60, 0x18, 0x90]
tm = cw_project.traceManager()
    
def do_the_ttvla(tm, ntraces=-1):
    if ntraces == -1:
        ntraces = int(tm.numTraces())
        
    print(ntraces)
    if ntraces > tm.numTraces():
        raise ValueError("Invalid ntraces")
        
    for g in range(0, 1):
        group = [(binascii.hexlify(texts[i]) == binascii.hexlify(bytes(fixedpy))) for i in range(g*ntraces, g*ntraces+ntraces)]
        trace = np.zeros((ntraces, tm.numPoints()))

        for n in range(g*ntraces, g*ntraces+ntraces):
            trace[n - g*ntraces][:] = tm.getTrace(n)
                
        testout = welch_ttest(group, trace)
        plot(testout)
        
    plot([0, tm.numPoints()], [-4.5, -4.5], 'r')
    plot([0, tm.numPoints()], [4.5, 4.5], 'r')

def welch_ttest(group, traces):
    # Compute Welch's t-statistic at each point in time
    # Here, group[] must only contain booleans (True/False)
    traces_true = traces[np.where(np.array(group))]
    traces_false = traces[np.where(~np.array(group))]
    
    if len(traces_true) == 0:
        traces_true  = np.array([[np.nan for _ in range(len(traces[0]))]])
        print("No true traces")
    if len(traces_false) == 0:
        traces_false = np.array([[np.nan for _ in range(len(traces[0]))]])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ttrace = scipy.stats.ttest_ind(traces_true, traces_false, axis=0, equal_var=False)[0]
        
    return np.nan_to_num(ttrace) 
    
do_the_ttvla(tm)
title("TVLA Results")
show()

In [ ]:
project = cw.openProject('projects/saml11')

 0: 2b7e151628aed2a6abf7158809cf4f3c
 1: a0fafe1788542cb123a339392a6c7605
 2: f2c295f27a96b9435935807a7359f67f
 3: 3d80477d4716fe3e1e237e446d7a883b
 4: ef44a541a8525b7fb671253bdb0bad00
 5: d4d1c6f87c839d87caf2b8bc11f915bc
 6: 6d88a37a110b3efddbf98641ca0093fd
 7: 4e54f70e5f5fc9f384a64fb24ea6dc4f
 8: ead27321b58dbad2312bf5607f8d292f
 9: ac7766f319fadc2128d12941575c006e
10: d014f9a8c9ee2589e13f0cc8b6630ca6